In [1]:
import torch

In [2]:
import requests
from pathlib import Path
from zipfile import ZipFile

data_path = Path("data/pizza_steak_sushi")

train_dir = data_path / "train"
test_dir = data_path / "test"

if data_path.is_dir():
    print("Folder sudah ada")
else: 
    with open("data/pizza_steak_sushi.zip", "wb") as f:
        res = requests.get(
            "https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi_20_percent.zip"
        )
        f.write(res.content)

    with ZipFile("data/pizza_steak_sushi.zip","r") as zf:
        zf.extractall(data_path)

Folder sudah ada


In [3]:
from torchvision.transforms import v2

train_transforms = v2.Compose([
        v2.ToImage(),
        v2.Resize(size=(64,64)),
        v2.TrivialAugmentWide(num_magnitude_bins=31),
        v2.ToDtype(torch.float32,scale=True)
    ])

test_transforms = v2.Compose([
        v2.ToImage(),
        v2.Resize(size=(64,64)),
        v2.ToDtype(torch.float32,scale=True)
    ])

In [6]:
# %%writefile going_modular/data_setup.py

from torchvision.datasets import ImageFolder
from torchvision.transforms import v2
from torch.utils.data import DataLoader
import os

NUM_WORKERS = os.cpu_count()
BATCH_SIZE = 32

def create_dataloaders(
    train_dir: str,
    test_dir: str,
    train_transforms: v2.Compose,
    test_transforms: v2.Compose,
    batch_size: int = BATCH_SIZE,
    num_workers: int = NUM_WORKERS,
):
    train_data = ImageFolder(root=train_dir,transform=train_transforms)
    test_data = ImageFolder(root=test_dir, transform=test_transforms)
    class_names = train_data.classes

    train_dataloader = DataLoader(
        dataset=train_data, batch_size=batch_size, num_workers=num_workers,shuffle=True,pin_memory=True
    )
    test_dataloader = DataLoader(
        dataset=test_data, batch_size=batch_size, num_workers=num_workers,pin_memory=True
    )
    


    return train_dataloader,test_dataloader,class_names

In [7]:
import os
from going_modular import data_setup

NUM_WORKERS = os.cpu_count()
BATCH_SIZE = 32

train_dir = data_path / "train"
test_dir = data_path / "test"

train_dataloader,test_dataloader,class_names = data_setup.create_dataloaders(train_dir=train_dir,test_dir=test_dir,train_transforms=train_transforms,test_transforms=test_transforms,batch_size=BATCH_SIZE,num_workers=NUM_WORKERS)

train_dataloader,test_dataloader,class_names

(<torch.utils.data.dataloader.DataLoader at 0x17d990729b0>,
 ['pizza', 'steak', 'sushi'])

In [9]:
train_dataloader.batch_size

32

In [7]:
%%writefile going_modular/model_builder.py

from torch import nn
class TinyVGG(nn.Module):
    def __init__(self,input_shape:int,hidden_units:int,output_shape:int):
        super().__init__()
        self.conv_block_1 = nn.Sequential(
            nn.Conv2d(input_shape, hidden_units, 3, 1, 1),
            nn.Conv2d(hidden_units, hidden_units, 3, 1, 1),
            nn.MaxPool2d(2),
        )

        self.conv_block_2 = nn.Sequential(
            nn.Conv2d(hidden_units, hidden_units * 2, 3, 1, 1),
            nn.Conv2d(hidden_units * 2, hidden_units * 2, 3, 1, 1),
            nn.MaxPool2d(2),
        )

        self.conv_block_3 = nn.Sequential(
            nn.Conv2d(hidden_units * 2, hidden_units, 3, 1, 1),
            nn.Conv2d(hidden_units, hidden_units, 3, 1, 1),
            nn.MaxPool2d(2),
        )

        self.classifer = nn.Sequential(
            nn.Flatten(),
            nn.Linear(hidden_units * 8 * 8, 64),
            nn.ReLU(),
            nn.Linear(64, 16),
            nn.ReLU(),
            nn.Linear(16,output_shape)
        )

    def forward(self,x):
        return self.classifer(self.conv_block_3(self.conv_block_2(self.conv_block_1(x))))

Overwriting going_modular/model_builder.py


In [8]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [9]:
from going_modular.model_builder import TinyVGG

model = TinyVGG(input_shape=3,hidden_units=32,output_shape=len(class_names)).to(device)
model

TinyVGG(
  (conv_block_1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_block_2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_block_3): Sequential(
    (0): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifer): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=2048, out_features=64, bias=True)
    (2): ReLU()
    (3): Linear(in_features=64, out_featu

In [10]:
from torchinfo import summary

summary(model,input_size=(32,3,64,64))

Layer (type:depth-idx)                   Output Shape              Param #
TinyVGG                                  [32, 3]                   --
├─Sequential: 1-1                        [32, 32, 32, 32]          --
│    └─Conv2d: 2-1                       [32, 32, 64, 64]          896
│    └─Conv2d: 2-2                       [32, 32, 64, 64]          9,248
│    └─MaxPool2d: 2-3                    [32, 32, 32, 32]          --
├─Sequential: 1-2                        [32, 64, 16, 16]          --
│    └─Conv2d: 2-4                       [32, 64, 32, 32]          18,496
│    └─Conv2d: 2-5                       [32, 64, 32, 32]          36,928
│    └─MaxPool2d: 2-6                    [32, 64, 16, 16]          --
├─Sequential: 1-3                        [32, 32, 8, 8]            --
│    └─Conv2d: 2-7                       [32, 32, 16, 16]          18,464
│    └─Conv2d: 2-8                       [32, 32, 16, 16]          9,248
│    └─MaxPool2d: 2-9                    [32, 32, 8, 8]           

In [ ]:
img,label = next(iter(train_dataloader))

img[0],label[0]

In [1]:
model.eval(),

with torch.inference_mode():
    y_logits = model(img[0].unsqueeze(0).to(device))
    print(y_logits)

NameError: name 'model' is not defined

In [2]:
from torchmetrics import Accuracy

acc_metric = Accuracy(task="multiclass", num_classes=3)

In [7]:
%%writefile going_modular/engine.py
import torch
from torch import nn
from torchmetrics import Metric


def train_step(
    model: nn.Module,
    data_loader: torch.utils.data.Dataset,
    loss_fn: nn.Module,
    acc_metric: Metric,
    optimizer: torch.optim.Optimizer,
    device,
):

    model.train()
    train_loss = 0
    acc_metric.reset(),

    for X, y in data_loader:
        X, y = X.to(device), y.to(device)
        y_logits = model(X)
        y_preds = torch.softmax(y_logits, dim=1)

        loss = loss_fn(y_logits, y)
        train_loss += loss
        acc_metric.update(y_preds, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    train_loss /= len(data_loader)
    train_acc = acc_metric.compute()

    return train_loss, train_acc


def test_step(
    model: nn.Module,
    data_loader: torch.utils.data.Dataset,
    loss_fn: nn.Module,
    acc_metric: Metric,
    device,
):

    model.eval()
    test_loss = 0
    acc_metric.reset(),

    with torch.inference_mode():
        for X, y in data_loader:
            X, y = X.to(device), y.to(device)
            y_logits = model(X)
            y_preds = torch.softmax(y_logits, dim=1)

            loss = loss_fn(y_logits, y)
            test_loss += loss
            acc_metric.update(y_preds, y)

        total_loss = test_loss / len(data_loader)
        test_acc = acc_metric.compute()

    return total_loss, test_acc


def train_model(
    model: nn.Module,
    train_dataloader: torch.utils.data.Dataset,
    test_dataloader: torch.utils.data.Dataset,
    loss_fn: nn.Module,
    acc_metric: Metric,
    optimizer: torch.optim.Optimizer,
    device,
    epochs=5,
):
    model.to(device)
    results = {"train_loss": [], "train_acc": [], "test_loss": [], "test_acc": []}

    for epoch in range(epochs):
        print(f"\nEpoch: {epoch}")
        train_loss, train_acc = train_step(
            model, train_dataloader, loss_fn, acc_metric, optimizer, device
        )
        test_loss, test_acc = test_step(
            model, test_dataloader, loss_fn, acc_metric, device
        )

        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

        print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
        print(f"Test Loss: {test_loss:.4f} | Test Acc: {test_acc:.4f}")
    return results

Overwriting going_modular/engine.py


In [8]:
from going_modular.engine import train_model

train_model()

TypeError: train_model() missing 7 required positional arguments: 'model', 'train_dataloader', 'test_dataloader', 'loss_fn', 'acc_metric', 'optimizer', and 'device'

In [2]:
%%writefile going_modular/utils.py
import torch
from torch import nn
from pathlib import Path

def save_model(model:nn.Module,target_dir:str,model_name:str):
    target_dir_path = Path(target_dir)
    target_dir_path.mkdir(parents=True,exist_ok=True)

    assert model_name.endswith("pth") or model_name.endswith("pt"), "Ekstensi model harus .pt atau .pth"
    model_save_path = target_dir / model_name

    torch.save(model_save_path,model.state_dict())


Overwriting going_modular/utils.py


In [ ]:
# %%writefile going_modular/train.py

from pathlib import Path
from going_modular.data_setup import create_dataloaders
from going_modular.model_builder import TinyVGG
from going_modular.engine import train_model
from going_modular.utils import save_model
from torchvision.transforms import v2
import torch
import os
from torchmetrics import Accuracy

device = "cuda" if torch.cuda.is_available() else "cpu"

data_path = Path("data/pizza_steak_sushi")

train_dir = data_path / "train"
test_dir = data_path / "test"

train_transforms = v2.Compose([
    v2.ToImage(),
    v2.Resize(size=(64,64)),
    v2.TrivialAugmentWide(num_magnitude_bins=31),
    v2.ToDtype(dtype=torch.float32,scale=True)
])

test_transforms = v2.Compose(
    [
        v2.ToImage(),
        v2.Resize(size=(64, 64)),
        v2.ToDtype(dtype=torch.float32, scale=True),
    ]
)

BATCH_SIZE = 32
NUM_WORKERS = os.cpu_count()
EPOCHS = 20

train_dataloader,test_dataloader, class_names = create_dataloaders(
                                                train_dir=train_dir,
                                                test_dir=test_dir,
                                                train_transforms=train_transforms,
                                                test_transforms=test_transforms,
                                                batch_size=BATCH_SIZE,
                                                num_workers=NUM_WORKERS)

model = TinyVGG(input_shape=3,hidden_units=32,output_shape=len(class_names)).to(device)


acc_metric = Accuracy(task="multiclass",num_classes=len(class_names)).to(device)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(),lr=0.001)


model_results = train_model(
    model=model,
    train_dataloader=train_dataloader,
    test_dataloader=test_dataloader,
    acc_metric=acc_metric,
    loss_fn=loss_fn,
    optimizer=optimizer,
    device=device,
    epochs=EPOCHS,
)

save_model(model, "model", "model_1.pt")
print(model_results)


Epoch: 0
Train Loss: 1.1040 | Train Acc: 0.3267
Test Loss: 1.1176 | Test Acc: 0.3133

Epoch: 1
Train Loss: 1.0883 | Train Acc: 0.3800
Test Loss: 0.9995 | Test Acc: 0.5733

Epoch: 2
Train Loss: 1.0374 | Train Acc: 0.4511
Test Loss: 0.9505 | Test Acc: 0.5200

Epoch: 3
Train Loss: 0.9964 | Train Acc: 0.4889
Test Loss: 0.9367 | Test Acc: 0.6067

Epoch: 4
Train Loss: 0.9373 | Train Acc: 0.5200
Test Loss: 1.0258 | Test Acc: 0.5133

Epoch: 5
Train Loss: 0.9900 | Train Acc: 0.4889
Test Loss: 0.9896 | Test Acc: 0.4667

Epoch: 6
Train Loss: 0.9931 | Train Acc: 0.4911
Test Loss: 1.0099 | Test Acc: 0.5667

Epoch: 7
Train Loss: 0.9390 | Train Acc: 0.5467
Test Loss: 1.0195 | Test Acc: 0.4400

Epoch: 8
Train Loss: 1.0206 | Train Acc: 0.4867
Test Loss: 0.9708 | Test Acc: 0.5600

Epoch: 9
Train Loss: 0.9473 | Train Acc: 0.5111
Test Loss: 0.9598 | Test Acc: 0.5067

Epoch: 10
Train Loss: 0.9300 | Train Acc: 0.5422
Test Loss: 0.9954 | Test Acc: 0.5000

Epoch: 11
